In [1]:
!pip install folium requests


In [2]:
import requests
import json
import pandas as pd

# URLs des données Vélib'
info_url = "https://velib-metropole-opendata.smovengo.cloud/opendata/Velib_Metropole/station_information.json"
status_url = "https://velib-metropole-opendata.smovengo.cloud/opendata/Velib_Metropole/station_status.json"

# Charger les données JSON
station_info = requests.get(info_url).json()
station_status = requests.get(status_url).json()

# Convertir en DataFrame
df_info = pd.json_normalize(station_info['data']['stations'])
df_status = pd.json_normalize(station_status['data']['stations'])

# Fusionner les deux
df = pd.merge(df_info, df_status, on='station_id')
df.head()


,station_id,stationCode_x,name,lat,lon,capacity,station_opening_hours_x,rental_methods,num_bikes_available,numBikesAvailable,num_bikes_available_types,num_docks_available,numDocksAvailable,is_installed,is_returning,is_renting,last_reported,stationCode_y,station_opening_hours_y
0,213688169,16107,Benjamin Godard - Victor Hugo,48.865983,2.275725,35,None,NaN,1,1,"[{'mechanical': 0}, {'ebike': 1}]",34,34,1,1,1,1748778484,16107,None
1,19179944124,40001,Hôpital Mondor,48.798922,2.453745,28,None,NaN,15,15,"[{'mechanical': 9}, {'ebike': 6}]",14,14,1,1,1,1748778597,40001,None
2,17278902806,44015,Rouget de L'isle - Watteau,48.778193,2.396302,20,None,NaN,19,19,"[{'mechanical': 10}, {'ebike': 9}]",0,0,1,1,1,1748778647,44015,None
3,36255,9020,Toudouze - Clauzel,48.879296,2.337360,21,None,[CREDITCARD],0,0,"[{'mechanical': 0}, {'ebike': 0}]",21,21,1,1,1,1748778257,9020,None
4,251039991,14111,Cassini - Denfert-Rochereau,48.837526,2.336035,25,None,[CREDITCARD],5,5,"[{'mechanical': 2}, {'ebike': 3}]",20,20,1,1,1,1748778738,14111,None


In [3]:
import folium

# Créer la carte centrée sur Paris
m = folium.Map(location=[48.8566, 2.3522], zoom_start=13)

# Ajouter les stations à la carte
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=6,
        popup=(
            f"<b>{row['name']}</b><br>"
            f"Vélos disponibles : {row['num_bikes_available']}<br>"
            f"Bornes libres : {row['num_docks_available']}"
        ),
        color='blue' if row['num_bikes_available'] > 0 else 'red',
        fill=True,
        fill_color='blue' if row['num_bikes_available'] > 0 else 'red'
    ).add_to(m)

m.save("velib_map.html")
m


In [6]:
from google.colab import files
files.download("velib_map.html")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>